# Python for Data Science
## Session 7
### Datasets – Pandas II

---

## Outline

1. Lambda and map

2. Series

2. Advanced data manipulation 

3. Handling missing data and data transformation 

---

## Pandas II 
### Lambda and map

A **Lambda** function is an anonymous tranformation. It is a **pure** function, since the output only depends on the input, with no side effects. ts syntax can be expressed as:
</p>

<center> <font color='blue'>lambda</font> arguments : expression</center> 

</p>

### Regular function

In [ ]:
def square(x):
    return x * x

### Equivalent lambda transformation

In [ ]:
square_lambda = lambda x: x * x

## Pandas II
### Lambda and map

Although we can used regular functions, as we saw in the last session, with **apply**, **lambda** tends to be the usual choice for one-off transformations.

In [ ]:
import pandas as pd
df = pd.DataFrame({'col': [1, 0, 4, 2, 5]})
df['col'] = df['col'].apply(lambda x: x * x)
df

## Pandas II

### Lambda and map

On the other side, **map**, is another option, originally created for **series** to perform one-to-one transformations. It is supposed to be faster than **apply**. For those familiar with **applymap**, note that this method will be deprecated in future versions.

In [ ]:
import numpy as np
import time

apply_values = []
map_values = []
for _ in range(10):
    df = pd.DataFrame({'col': np.random.randn(1000000)})
    ts = time.time()
    df['col'].apply(lambda x: x * x)
    apply_values.append(time.time() - ts)
    
    ts = time.time()
    df['col'].map(lambda x: x * x)
    map_values.append(time.time() - ts)
    
print(f"Apply: it took for 1M values {np.mean(apply_values)} seconds.")
print(f"Map: it took for 1M values {np.mean(map_values)} seconds.")

## Pandas II
### series

Let's jump into **series**. As we roughly mentioned in the last session, they are mainly used for time series and 1D observations. 

In [ ]:
s = pd.Series([np.random.randint(0, 100 + 1) for _ in range(100)])
s.map(lambda x: x - 100).head(4) # s.apply(lambda x: x - 100) 

In [ ]:
s = s - 100 # we can also perform easy operations over the whole series
s.head(4)

In [ ]:
s = pd.Series([1, 2, 3, 4])
s.map({1: 'A', 2: 'B'}) # replace values using a dictionary with Map

## Pandas II
### Series

Another cool method we can call is **rolling** which performs a sliding window over the whole series

In [ ]:
s = pd.Series([1, 20, 300, 4000])
s.rolling(window=2).mean()  # 2-period rolling mean

## Pandas II
### Advanced data manipulation

Some methods we can use with **series** and **dataframes** are:

- value_counts
- drop_duplicates

Values counts provide a fast way of exploring and handling your data

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor', 'Laura'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'doctoral', 'master']
})
df['program'].value_counts()

We can also drop duplicates based on all the columns

In [ ]:
df.drop_duplicates()['program'].value_counts()

We can also remove duplicates based on specific columns

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Ludmila'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'doctoral'],
    'year': [2019, 2017, 2021, 2023, 2017]
})
df.drop_duplicates(subset=['student', 'program'])

We can also select which one to remove based on its appearance, **first** or **last**, or even drastically remove all of them by setting **keep** to **False**:

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Ludmila'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'doctoral'],
    'year': [2019, 2017, 2021, 2023, 2017]
})
df.drop_duplicates(subset=['student', 'program'], keep='last') # keep = False will remove all

As we mentioned in the last session, we can modify a column type. This can be an option for optimising resources. What we did not see is that we can simply call it once to modify multiple at the same time:

In [ ]:
df.astype({'year': np.int16, 'student': 'S10'}).info() 

It may happen that some errors occur

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura'],
    'program': ['doctoral', 'master'],
    'id': ['1232', 'ABDD'],
    'year': [2019, 2017]
})
# ignore will leave as it was
df.astype({'id': float}, errors='ignore') 

If you want to still change the column type, and handle them in a specific way, you can use the following:

In [ ]:
pd.to_numeric(df['id'], errors='coerce') # Ideally, you can use to_numeric

## Pandas II
### Advanced data manipulation

Let's do some exercices with what we learned so far. The Iris dataset can be found in: 

https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

Exercise:

- Series
    - Create a **lambda** function that classifies flowers based on their petal length (e.g., "short" for petal lengths less than 3 cm and "long" for others). Apply this function to the PetalLengthCm column using **apply**.
    - Use **map** to convert the *Species* column into numeric values, e.g. 0 for *Iris-setosa*, 1 for *Iris-versicolor*, 2 for *Iris-virginica*.
- DataFrame
    - Use **value_counts** on the *Species* column to count how many entries belong to each species.
    - Use **drop_duplicates** to remove any duplicate rows from the dataset based on SepalLengthCm and PetalLengthCm.
    - Use **astype** to convert the *SepalLengthCm* column to a string type, then back to float type (if there are any errors, handle them gracefully).
    - Save the modified DataFrame to a CSV file ensuring the index is included (use **to_csv**).

In [19]:
# Additional information to succeed in the exercises
import pandas as pd
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
# Since the dataset does not come with header... we set up the columns with the above information
df = pd.read_csv(url, header=None, names=columns)

In [20]:
classifier=lambda x: 'short' if x<3 else 'long'
df['PetalLengthCm'] = df['PetalLengthCm'].apply(classifier)

In [21]:
df['Species'].map(lambda x: 0 if x=='Iris-setosa' else (1 if x=='Iris-versicolor' else 2 if x=='Iris-virginica' else None))

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: Species, Length: 150, dtype: int64

In [23]:
print(df['Species'].value_counts())
df.drop_duplicates(subset=['SepalLengthCm','PetalLengthCm'])

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,short,0.2,Iris-setosa
1,4.9,3.0,short,0.2,Iris-setosa
2,4.7,3.2,short,0.2,Iris-setosa
3,4.6,3.1,short,0.2,Iris-setosa
4,5.0,3.6,short,0.2,Iris-setosa
5,5.4,3.9,short,0.4,Iris-setosa
8,4.4,2.9,short,0.2,Iris-setosa
11,4.8,3.4,short,0.2,Iris-setosa
13,4.3,3.0,short,0.1,Iris-setosa
14,5.8,4.0,short,0.2,Iris-setosa


In [ ]:
df.astype({'SepalLengthCm':str}, errors='ignore').info()
df.astype({'SepalLengthCm':float}, errors='ignore').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    object 
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    object 
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(2), object(3)
memory usage: 6.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    object 
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(3), object(2)
memory usage: 6.0+ KB


In [ ]:
df.to_csv(r'Dataframe.csv')

## Pandas II
### Advanced data manipulation

As we saw last session, **groupby** is a pretty handy tool. Another useful tool is **pivot_table**. It provides  statistics of a more extensive data table. 

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'doctoral'],
    'grades': ['B', 'A', 'F', 'C', 'C', 'B', 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 8, 8, 9]
})
df

In [ ]:
df.pivot_table(values='marks', index='program') # it will by default use mean

In [ ]:
df.pivot_table(values='marks', index='program', aggfunc='max')

In [ ]:
# here it does not make sense to use index = student, 
# since we don't have multile entries per student
df.pivot_table(values='marks', index='student', aggfunc=['sum', 'mean'])

## Pandas II
### Advanced data manipulation

Another interesting method we can use within **Pandas** is **query**. It can help us exploring in a neat manner:


In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'doctoral'],
    'grades': ['B', 'A', 'F', 'C', 'C', 'B', 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 7.8, 8, 9]
})

In [ ]:
df.query('marks >= 8 and grades in ["B", "A"]') 

It's equivalent without query

In [ ]:
df[(df['marks'] >= 8) & (df['grades'].isin(['B', 'A']))]

## Pandas II
### Advanced data manipulation

Similar to SQL, we can also **merge** and **join** dataframes based on a specific column:

<table border="1" style="border-collapse: collapse; width: 90%;">
  <tr>
    <th style="text-align: center;">Operation</th>
    <th style="text-align: center;">Ideal For</th>
    <th style="text-align: center;">Joins on</th>
    <th style="text-align: center;">Syntax</th>
  </tr>
  <tr>
    <td style="text-align: center;"><b>merge</b></td>
    <td style="text-align: center;">Column-based joins</td>
    <td style="text-align: center;">Columns or indexes</td>
    <td style="text-align: center;"><code>pd.merge(df1, df2, on='col')</code></td>
  </tr>
  <tr>
    <td style="text-align: center;"><b>join</b></td>
    <td style="text-align: center;">Index-based joins</td>
    <td style="text-align: center;">Index alignment</td>
    <td style="text-align: center;"><code>df1.join(df2, on='index_col')</code></td>
  </tr>
</table>


In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', 'F', 'C', 'C', 'B', 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 7.8, 8, 9]
})


program_info = pd.DataFrame({
    'program': ['doctoral', 'master', 'post-doctoral', 'pre-doctoral'],
    'duration': ['5 years', '2 years', '3 years', '1 year'],
})

In [ ]:
pd.merge(df, program_info, on='program', how='inner') # merge them using inner

In [ ]:
pd.merge(df, program_info, on='program', how='left') # merge them using left

In [ ]:
pd.merge(df, program_info, on='program', how='right') # merge them using left

In [ ]:
pd.merge(df, program_info, on='program', how='outer') # merge them using outer

We can do similar with **join** as we do for **merge**.

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', 'F', 'C', 'C', 'B', 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 7.8, 8, 9]
})


program_info = pd.DataFrame({
    'program': ['doctoral', 'master', 'post-doctoral', 'pre-doctoral'],
    'duration': ['5 years', '2 years', '3 years', '1 year'],
})

program_info.set_index('program', inplace=True)
df.join(program_info, on='program', how='inner')

## Pandas II
### Advanced data manipulation

Another thing we can do with **pandas** consists in setting up, accessing, and slicing multi-level indices.


Setting up multi-level index:

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', 'F', 'C', 'C', 'B', 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 7.8, 8, 9]
})


df.set_index(['program', 'grades'], inplace=True)
df

Once, set up, we can use **loc**:

In [ ]:
df.loc[('doctoral', 'C')] # we may have some warning here

In [ ]:
df.loc['doctoral'] # we can access by the top level

In [ ]:
idx = pd.IndexSlice
idx

In [ ]:
idx['doctoral', 'B':'C']
df.loc[idx['doctoral', 'B':'C'], :] # this is gonna raise an error

In [ ]:
df = df.sort_index() # we sort 
df.loc[idx['doctoral', 'B':'C'], :] # it's no longer raising an error

And, of course, we can reset index:

In [ ]:
df.reset_index(inplace=True)
df

## Pandas II
### Handling Missing Data and Data Transformation

From here, we will quickly see other ways of replacing missing data and string manipulation.

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', np.nan, 'C', 'C', np.nan, 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 7.8, 8, 9]
})

Forward filling, using the current value to fill the next missing one

In [ ]:
df['new_grades'] = df['grades'].ffill()
df

Backward filling, using the next value to fill the missing one

In [ ]:
df['new_grades'] = df['grades'].bfill()
df

In [ ]:
df['grades'].fillna('F') # of course, we can use the one we learned in session 6

Another possibility is to provide a grade map and fill the missing values using other columns, something that can certainly happen when manipulatin data.

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', np.nan, 'C', 'C', np.nan, 'B', 'A'],
    'marks':  [8, 10, 4, 5, 5, 7, 8, 9]
})

In [ ]:
grade_map = {10: 'A', 9: 'A', 8: 'B', 7: 'B', 6: 'C', 5: 'C', 4: 'F', 3: 'F'}
df['grades_numeric'] = df['marks'].map(grade_map) # we can fill missing values relying on other columns
df

Another way is to interpolate:

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', np.nan, 'C', 'C', 'B', 'B', np.nan],
    'marks':  [8, 10, np.nan, 5, 5, 7, 8, np.nan]
})
df['marks'].interpolate() # this won't work for categorical data like letter grades

In [ ]:
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor'],
    'program': ['doctoral', 'master', 'post-doctoral', 'doctoral', 'master', 'doctoral', 'post-doctoral', 'bachelor'],
    'grades': ['B', 'A', np.nan, 'C', 'C', 'B', 'B', np.nan],
    'marks':  [8, 10, np.nan, 5, 5, 7, 8, np.nan]
})

In [ ]:
# Like this we will replace grades with the most common value per group
new_values = df.groupby('program')['grades'].apply(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'F'))
new_values # returns a Series

In [ ]:
second_level_index = new_values.index.get_level_values(1) # to access the second level index

In [ ]:
df.loc[second_level_index, 'grades'] = new_values.values

In [ ]:
df

## Pandas II
### Handling Missing Data and Data Transformation

Let's do a few string manipulations with pandas

Extracting the first letter of each student's name

In [7]:
import pandas as pd
df = pd.DataFrame({
    'student': ['Ludmila', 'Laura', 'Gerardo', 'Antonio', 'Manuel', 'Frank', 'Marco', 'Victor']
})

df['first_letter'] = df['student'].str.extract(r'(^.)')
df

,student,first_letter
0,Ludmila,L
1,Laura,L
2,Gerardo,G
3,Antonio,A
4,Manuel,M
5,Frank,F
6,Marco,M
7,Victor,V


In [8]:
df['first_letter'].str.lower() # we can change them to lower, instead of capitals (upper)

0    l
1    l
2    g
3    a
4    m
5    f
6    m
7    v
Name: first_letter, dtype: object

In [9]:
# replacing any letter by *
df['student'].str.replace(r'[aeiouAEIOU]', '*', regex=True) # using regex on the back

0    L*dm*l*
1      L**r*
2    G*r*rd*
3    *nt*n**
4     M*n**l
5      Fr*nk
6      M*rc*
7     V*ct*r
Name: student, dtype: object

In [10]:
# check if it includes a specific letter
df['contains_a'] = df['student'].str.contains(r'a', case=False)
df

,student,first_letter,contains_a
0,Ludmila,L,True
1,Laura,L,True
2,Gerardo,G,True
3,Antonio,A,True
4,Manuel,M,True
5,Frank,F,True
6,Marco,M,True
7,Victor,V,False


In [11]:
# Split columns into multiple ones, like first name and last name
df = pd.DataFrame({
    'professor': ['Ludmila Kuncheva', 'Antonio Torralba', 'Manuel Gonzalez', 'Bastian Leibe']
})
    
df[['first_name', 'last_name']] = df['professor'].str.split(' ', expand=True)
df

,professor,first_name,last_name
0,Ludmila Kuncheva,Ludmila,Kuncheva
1,Antonio Torralba,Antonio,Torralba
2,Manuel Gonzalez,Manuel,Gonzalez
3,Bastian Leibe,Bastian,Leibe


In [12]:
# Using regex again, extracting specific information from a column
df = pd.DataFrame({
    'student': ['Ludmila 1960', 'Laura 1980', 'Gerardo 1970', 'Antonio 1970', 'Manuel 1960']
})

# Extract numbers from the 'student' column
df['years'] = df['student'].str.extract(r'(\d+)').astype(int)
df

,student,years
0,Ludmila 1960,1960
1,Laura 1980,1980
2,Gerardo 1970,1970
3,Antonio 1970,1970
4,Manuel 1960,1960


## Pandas II
### Handling Missing Data and Data Transformation

Let's practica a little bit here, take home what you did not solve today.

Exercises:

1. Create a new column called **professor_initials** that stores the initials of each professor's first and last names. Use the following data:

In [13]:
data = {
    'professor': ['Ludmila Kuncheva', 'Antonio Torralba', 'Manuel Gonzalez', 'Bastian Leibe'],
    'department': ['Computer Science', 'Computer Vision', 'AI & Robotics', 'Autonomous Systems'],
    'age': [45, 50, 47, 38]
}

df = pd.DataFrame(data)
name=df['professor'].str.split(' ', expand=True)
df['professor_initials'] = name[0].str[0]+name[1].str[0]

In [14]:
df

,professor,department,age,professor_initials
0,Ludmila Kuncheva,Computer Science,45,LK
1,Antonio Torralba,Computer Vision,50,AT
2,Manuel Gonzalez,AI & Robotics,47,MG
3,Bastian Leibe,Autonomous Systems,38,BL


2. Given the dataframe below. Use **join** to combine this new DataFrame with the original one based on the professor column.

In [15]:
courses_data = {
    'professor': ['Ludmila Kuncheva', 'Antonio Torralba', 'Manuel Gonzalez', 'Bastian Leibe'],
    'courses': ['Machine Learning', 'Computer Vision', 'AI Programming', 'Self-Driving Cars']
}
df_courses = pd.DataFrame(courses_data)

df.set_index('professor', inplace=True)
df_courses.join(df,on='professor',how='inner')

,professor,courses,department,age,professor_initials
0,Ludmila Kuncheva,Machine Learning,Computer Science,45,LK
1,Antonio Torralba,Computer Vision,Computer Vision,50,AT
2,Manuel Gonzalez,AI Programming,AI & Robotics,47,MG
3,Bastian Leibe,Self-Driving Cars,Autonomous Systems,38,BL


3. Combine the original df and df_courses DataFrames.

In [16]:
data = {
    'professor': ['Ludmila Kuncheva', 'Antonio Torralba', 'Manuel Gonzalez', 'Bastian Leibe'],
    'department': ['Computer Science', 'Computer Vision', 'AI & Robotics', 'Autonomous Systems'],
    'age': [45, 50, 47, 38]
}

df = pd.DataFrame(data)

courses_data = {
    'professor': ['Ludmila Kuncheva', 'Antonio Torralba', 'Manuel Gonzalez', 'Bastian Leibe'],
    'courses': ['Machine Learning', 'Computer Vision', 'AI Programming', 'Self-Driving Cars']
}
df_courses = pd.DataFrame(courses_data)

df.merge(df_courses, on='professor', how='inner')

,professor,department,age,courses
0,Ludmila Kuncheva,Computer Science,45,Machine Learning
1,Antonio Torralba,Computer Vision,50,Computer Vision
2,Manuel Gonzalez,AI & Robotics,47,AI Programming
3,Bastian Leibe,Autonomous Systems,38,Self-Driving Cars


4. In the professor column, create a new column professor_last_name by extracting the last name of each professor using string operations.

In [17]:
df['professor_last_name']=df['professor'].str.split(' ',expand=True)[1]
df

,professor,department,age,professor_last_name
0,Ludmila Kuncheva,Computer Science,45,Kuncheva
1,Antonio Torralba,Computer Vision,50,Torralba
2,Manuel Gonzalez,AI & Robotics,47,Gonzalez
3,Bastian Leibe,Autonomous Systems,38,Leibe


## Pandas II

### Summary

Today we saw deeper ways of manipulating **dataframes** and **series**, from applying functions on columns, merging / joining multiple datasets, replacing missing values, to finally work with strings using regex in the back.

We hope you enjoyed this session, see you next in session 8!